In [122]:
import os, datetime
from imapclient import IMAPClient
from utils import *
from dotenv import load_dotenv
load_dotenv()

USERNAME = os.getenv("user")
PASSWORD = os.getenv("pass")

In [123]:
server = IMAPClient('imap.gmail.com', ssl=True, use_uid=True)
server.login(USERNAME, PASSWORD)
select_info = server.select_folder('INBOX')
print('%d messages in INBOX' % select_info[b'EXISTS'])

6864 messages in INBOX


In [124]:
list_of_ids = get_ids(server=server, from_date=datetime.date(2025, 8, 1)) #year,month,day
server.logout()
print(list_of_ids[:5])

[16774, 16775, 16776, 16777, 16778]


In [129]:
from concurrent.futures import ThreadPoolExecutor

def fetch_batch(ids):
    with IMAPClient('imap.gmail.com', ssl=True) as server:
        server.login(USERNAME, PASSWORD)
        server.select_folder('INBOX')
        msg_data = server.fetch(ids, ['RFC822', 'ENVELOPE'])
        server.logout()
        return msg_data
batch_size = 10
id_chunks = [list_of_ids[i:i+batch_size] for i in range(0, len(list_of_ids), batch_size)]

In [130]:
with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_batch, id_chunks))

    # Combine all results
#final_result = {k: v for batch in results for k, v in batch.items()}

In [63]:
#msg_id = list_of_ids[:2]
msg_data = server.fetch(list_of_ids, ['RFC822', 'ENVELOPE'])

In [64]:
msg_id = msg_data[list_of_ids[1]][b'SEQ']
raw_email = msg_data[list_of_ids[1]][b'RFC822']
envelope = msg_data[list_of_ids[1]][b'ENVELOPE']

#print(msg_data[list_of_ids[1]].keys()) ====>> dict_keys([b'SEQ', b'RFC822', b'ENVELOPE'])

In [65]:
sender, name, email_addr, body = get_msg_data(server, list_of_ids)

# Print the result
print("Sender Name:", name)
print("Sender Email:", email_addr)
#print("Body:\n", body)

Sender Name: Kaggle
Sender Email: no-reply@kaggle.com


In [66]:
sender

Envelope(date=datetime.datetime(2025, 8, 7, 4, 54, 47), subject=b'Competition Launch: NeurIPS 2025 - Google Code Golf Championship', from_=(Address(name=b'Kaggle', route=None, mailbox=b'no-reply', host=b'kaggle.com'),), sender=(Address(name=b'Kaggle', route=None, mailbox=b'no-reply', host=b'kaggle.com'),), reply_to=(Address(name=b'Kaggle', route=None, mailbox=b'no-reply', host=b'kaggle.com'),), to=(Address(name=None, route=None, mailbox=b'sathwikmethari', host=b'gmail.com'),), cc=None, bcc=None, in_reply_to=None, message_id=b'<4af014169424efa8b29b4806d849e89660d20848-20364489-111676857@google.com>')

In [67]:
import re

def has_html(text):
    return bool(re.search(r'<[a-z/][^>]*>', text, re.IGNORECASE))

# Example usage:
print(has_html(body)) # True
print(has_html("This is a plain string.")) # False

False
False


In [68]:
import sys
size_in_bytes=sys.getsizeof(raw_email)

size_in_mb=size_in_bytes / (1024 * 1024)
print(f"Size in MB: {size_in_mb}")

Size in MB: 0.04775714874267578


In [69]:
import pandas as pd

In [79]:
df = pd.DataFrame(msg_data)
df=df.T
#df.to_csv(output_file, index=False)

In [80]:
df.head()

,b'SEQ',b'RFC822',b'ENVELOPE'
16774,6665,b'Delivered-To: sathwikmethari@gmail.com\r\nRe...,"Envelope(date=datetime.datetime(2025, 8, 1, 1,..."
16775,6666,b'Delivered-To: sathwikmethari@gmail.com\r\nRe...,"Envelope(date=datetime.datetime(2025, 8, 1, 1,..."
16776,6667,b'Delivered-To: sathwikmethari@gmail.com\r\nRe...,"Envelope(date=datetime.datetime(2025, 8, 1, 3,..."
16777,6668,b'Delivered-To: sathwikmethari@gmail.com\r\nRe...,"Envelope(date=datetime.datetime(2025, 8, 1, 5,..."
16778,6669,"b""Delivered-To: sathwikmethari@gmail.com\r\nRe...","Envelope(date=datetime.datetime(2025, 8, 1, 9,..."


In [72]:
r,c =df.shape

In [73]:
c

3